# Building blocks of Clojure

## Metadata

In [ ]:
; Using the with-meta function to add metadata to a map
(def untrusted (with-meta 
    {:command "delete-table" :subject "users"}
    {:safe false :io true})) ; Metadata

untrusted

In [ ]:
; Using the '^' macro to add metadata to a map

(def untrusted 
    ^{:safe false :io true} 
    {:command "delete-table" :subject "users"})

untrusted

In [ ]:
; If using the '^' macro with a list, 
; the metadata won't take effect at runtime

(def untrusted-not-working 
    ^{:safe false :io true} 
    (hash-map :command "delete-table" :subject "users"))


In [ ]:
; Metadata doesn't affect value equality

(def trusted {:command "delete-table" :subject "users"}) 
(= trusted untrusted) 

In [ ]:
; Using the meta function to examine metadata
(meta untrusted)


In [ ]:
; When new values are created from those that have metadata, 
; the metadata is copied over to the new data

(def still-untrusted (assoc untrusted :complete? false))
(meta still-untrusted)


In [ ]:
; Functions and macros can also be defined with metadata

(defn ^{:safe true 
        :console true
        :doc "testing metadata for functions"} 
    testing-meta
    [] 
    (println "Hello from meta!"))


In [ ]:
; Trying to use the meta function directly to check that 
; the metadata was set correctly won't work

(meta testing-meta)

In [ ]:
; To access the metadata,  pass the testing-meta var to the meta function.
(meta (var testing-meta))

Defining a function without Java type hints

In [ ]:
; Setting a warning to alert when "Reflection" is used
(set! *warn-on-reflection* true) 

(defn string-length [x] (.length x))


In [ ]:
; Function call without type hints (slow)

(time (reduce + (map string-length (repeat 10000 "12345"))))

Defining a function with Java type hints

In [ ]:
(defn fast-string-length [^String x] (.length x)) 


In [ ]:
; Function call with type hints (fast)

(time (reduce + (map fast-string-length (repeat 10000 "12345"))))

In [ ]:
; The type hints of args is stored as metadata

(meta (first (first (:arglists (meta #'fast-string-length))))) 


Type hinting primitive types

In [ ]:
; Getting the Java Class name of BigDecimal

(defn array-type [klass]
    (.getName (class (make-array klass 0)))) 

(array-type BigDecimal)

In [ ]:
; Using the name "[Ljava.math.BigDecimal;" as type hint

(def bigdec-arr 
    ^"[Ljava.math.BigDecimal;" 
    (into-array BigDecimal [1.0M]))


## Java exceptions: try and throw


In [ ]:
; A function that calculates the average of a collection of numbers

(defn average [numbers]
    (let [total (apply + numbers)]
        (/ total (count numbers))))

In [ ]:
; Calling the average function with an empty sequence throws an exception

(average [])

In [ ]:
; Wrapping the last function with a try/catch expression

(defn safe-average [numbers]
    (let [total (apply + numbers)]
        (try
            (/ total (count numbers))
        (catch ArithmeticException e 
            (println "Divided by zero!")
        0))))

(safe-average [])

In [ ]:
; The general form of using try/catch/finally is straightforward:

"""
(try expr* catch-clause* finally-clause?)
"""

In [ ]:
; Using several catch and a finally clauses

(try
    (print "Attempting division... ")
    (/ 1 0)
(catch RuntimeException e "Runtime exception!") 
(catch ArithmeticException e "DIVIDE BY ZERO!")  
(catch Throwable e "Unknown exception encountered!") 
(finally 
    (println "done.")))


In [ ]:
; The finally clause can be used without catch clauses
(try
    (print "Attempting division... ")
    (/ 1 0)
(finally
    (println "done.")))


In [ ]:
; Throwing exceptions
(throw (Exception. "this is an error!"))

## Functions

Function definitions

In [ ]:
(use '[clojure.repl :only (doc)])

In [ ]:
; Syntax of the defn macro
"""
(defn function-name
    doc-string?
    metadata-map?
    [parameter-list*]
    conditions-map?
    body-expressions*) 
"""

In [ ]:
; A basic function definition

(defn total-cost [item-cost number-of-items]
    (* item-cost number-of-items))

In [ ]:
; The defn macro expands the last function definition as follows

(def total-cost (fn [item-cost number-of-items]
    (* item-cost number-of-items)))


In [ ]:
; Adding docstring at function definition
(defn total-cost 
    "return line-item total of the item and quantity provided" 
    [item-cost number-of-items]
    (* item-cost number-of-items))

(doc total-cost)

Adding metadata to functions

In [ ]:
; Adding metadata without using ^
(meta (defn myfn-attr-map {:a 1} [])) 

In [ ]:
; Adding metadata with ^
(meta (defn ^{:a 1} myfn-metadata []))

In [ ]:
; When adding duplicate metadata, the last version is used
(meta (defn ^{:a 1} myfn-both {:a 2 :b 3} [])) 

In [ ]:
; When adding duplicate docstrings, the last version is used
(meta (defn ^{:a 1 :doc "doc 1"} myfn-redundant-docs "doc 2" {:a 2 :b 3 :doc "doc 3"} [])) 

Adding conditional to functions

In [ ]:
; Adding pre and post conditionals to functions
(defn item-total [price quantity discount-percentage]
    {:pre [(> price 0) (> quantity 0)]
     :post [(> % 0)]}
    (->> (/ discount-percentage 100)
         (- 1)
         (* price quantity)
         float))


In [ ]:
; Testing pre conditionals with a valid input
(item-total 100 2 10)-

In [ ]:
; Testing pre conditionals with an invalid input
(item-total 100 -2 10)

In [ ]:
; Testing the post conditional with an invalid input
(item-total 100 2 110)


Multiple 'arity' of functions

In [ ]:
; The general form of overloaded functions is as follows
"""
(defn function-name 
    ;; Note that each argument+body pair is enclosed in a list.
    ([arg1] body-executed-for-one-argument-call)
    ([arg1 arg2] body-executed-for-two-argument-call)
    ;; More cases may follow.)
"""

In [ ]:
; Example of an overloaded function
(defn total-cost 
    ([item-cost number-of-items]
        (* item-cost number-of-items))
    ([item-cost]
        (total-cost item-cost 1))) ; Calling the 2-ary version from the 1-ary one


Variadic functions

In [ ]:
; The general form of variadic functions is as follows
"""
(defn name-of-variadic-function [param-1 param-2 & rest-args]
    (body-of-function))
"""

In [ ]:
; Using the & symbol to define a variadic function

(defn total-all-numbers [& numbers]
    (apply + numbers))

In [ ]:
; A variadic function with other nonvariadic arities
(defn many-arities
    ([] 0)
    ([a] 1)
    ([a b c] 3)
    ([a b c & more] "variadic"))


In [ ]:
; >Testing it with 0 args
(many-arities)

In [ ]:
; Testing it with 1 arg
(many-arities "one argument")

In [ ]:
; Testing it with 2 args (this case is not defined)
(many-arities "two" "arguments")

In [ ]:
; Testing it with 3 args
(many-arities "three" "argu-" "ments")

In [ ]:
; Testing it with more than 3 args
(many-arities "many" "more" "argu-" "ments")

Self-Recursive functions

In [ ]:
; A recursive function that will blow the stack with big args
(defn count-down [n]
    (when-not (zero? n)
        (when (zero? (rem n 100)) 
            (println "count-down:" n))
        (count-down (dec n)))) 

;(count-down 100000) ; This will throw StackOverflowError
(count-down 500)

In [ ]:
; Rewriting the last function with recur, which won't blow the stack
(defn count-downr [n]
    (when-not (zero? n)
        (if (zero? (rem n 100))
            (println "count-down:" n))
        (recur (dec n))))

;(count-downr 100000) ; This won't throw StackOverflowError, but will kill the jupyter kernel. Try it at the REPL
(count-downr 500)


Mutually recursive functions

In [ ]:
; Mutually recursive functions that can blow the stack
(declare hat)
(defn cat [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "cat:" n))
        (hat (dec n)))) ; Calling hat

(defn hat [n]
    (when-not (zero? n)
        (if (zero? (rem n 100))
            (println "hat:" n))
        (cat (dec n)))) ; Calling cat

; (cat 5000000) ; This will throw StackOverflowError
(cat 500)

In [ ]:
; Calling mutually recursive functions with the trampoline function

(declare hatt)
(defn catt [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "catt:" n))
        (fn [] (hatt (dec n)))))

(defn hatt [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "hatt:" n))
        (fn [] (catt (dec n)))))

; (trampoline catt 5000000) ; This won't throw StackOverflowError, but will kill the jupyter kernel. Try it at the REPL
(trampoline catt 500)


In [ ]:
; Internal workings of the trampoline function
(defn trampoline
    ([f]
        (let [ret (f)]
            (if (fn? ret) ; fn? means 'Is my argument a function?'
                (recur ret)
            ret)))
    ([f & args]
    (trampoline (fn [] (apply f args)))))

Calling functions

In [ ]:
; Standard function call
(+ 1 2 3 4 5)

In [ ]:
; Applying a function to a sequence of args with apply

(def list-of-expenses [39.95M 39.95M 39.95M 39.95M 39.95M]) 
(apply + list-of-expenses) 


Higher order functions

every?

In [ ]:
;Checking if all sequence elements evaluate to True with every?

(def bools [true true true false false])
(every? true? bools) 

some

In [ ]:
;Checking if at least 1 sequence element evaluates to True with some

(def bools [true true true false false])
(some true? bools) 


constantly

In [ ]:
; Returning a fixed constant with constantly

(def two (constantly 2)) ; same as (defn two [& more] 2)

In [ ]:
(two 1)

In [ ]:
(two :a :b :c)


complement

In [ ]:
; A function that ckecks is the 1st arg is greater than teh 2nd
(defn greater? [x y]
    (> x y))

(greater? 10 5)

In [ ]:
; Taking the original function and returning the logically opposite value with complement
(def smaller? (complement greater?))

(smaller? 5 10)

comp

In [ ]:
; Composing functions with comp

(def opp-zero-str (comp str not zero?)) ; Checks if the argument is nonzero

(opp-zero-str 1)

partial

In [ ]:
; A function that checks if some number is above a threshold
(defn above-threshold? [threshold number]
    (> number threshold))


In [ ]:
; Partially applying the above-threshold? function with threshold 5, 
; Then using it to filter a sequence

(filter (partial above-threshold? 5) [ 1 2 3 4 5 6 7 8 9])

memoize

In [2]:
; Calling an artificially slow function (sleeps 1s before computing the result)
(defn slow-calc [n m]
    (Thread/sleep 1000)
    (* n m))

(time (slow-calc 5 7))


"Elapsed time: 1000.8107 msecs"


35

In [4]:
; Creating a new function that uses the memoize function to cache past computations

(def fast-calc (memoize slow-calc))

#'user/fast-calc

In [5]:
; Calling it the 1st time (slow)

(time (fast-calc 5 7))

"Elapsed time: 1000.4732 msecs"


35

In [6]:
; Calling it the 2nd time (fast)

(time (fast-calc 5 7))

"Elapsed time: 0.1219 msecs"


35

Writing higher order functions

In [9]:
; A mapping that will be sorted in various ways

(def users 
    [{:username "kyle"
      :firstname "Kyle"
      :lastname "Smith"
      :balance 175.00M ; Use BigDecimals for money!
      :member-since "2009-04-16"}
     {:username "zak"
      :firstname "Zackary"
      :lastname "Jones"
      :balance 12.95M
      :member-since "2009-02-01"}
     {:username "rob"
      :firstname "Robert"
      :lastname "Jones"
      :balance 98.50M
      :member-since "2009-03-30"}])


#'user/users

In [27]:
; Sort all the sequence by the user key

(sort-by username users)

({:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"})

In [40]:
; The last function sorts like the following functions

(sort (map username users))

("kyle" "rob" "zak")

In [31]:
;A HOF that sorts a sequence by an arbitrary key

(defn sorter-using [ordering-fn]
    (fn [collection]
        (sort-by ordering-fn collection)))


#'user/sorter-using

In [30]:
; A function that selects the lastname and firstname as keys

(defn lastname-firstname [user]
    [(user :lastname) (user :firstname)])

#'user/lastname-firstname

In [29]:
; A function that selects the balance as key

(defn balance [user] 
    (user :balance))

#'user/balance

In [33]:
; A function that selects the username as key

(defn username [user] 
    (user :username))

#'user/username

In [34]:
; Using the sorter HOF and the balance key function
; to get the users sorted by balance

(def poorest-first 
    (sorter-using balance))

(poorest-first users)

({:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"})

In [35]:
; Using the sorter HOF and the username key function
; to get the users sorted by name

(def alphabetically 
    (sorter-using username))

(alphabetically users)

({:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"})

In [36]:
; Using the sorter HOF and the lastname-firstname key function
; to get the users sorted by last name

(def last-then-firstname 
    (sorter-using lastname-firstname))

(last-then-firstname users)

({:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"} {:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"})

In [39]:
; The last function sorts like the following functions

(sort (map lastname-firstname users))

(["Jones" "Robert"] ["Jones" "Zackary"] ["Smith" "Kyle"])

Anonymous functions

In [43]:
; Creating an anonymous function with the fn macro

(fn [item-cost number-of-items]
    (* item-cost number-of-items))

#function[user/eval4171/fn--4172]

In [46]:
; Using map with an anonymous function to get the join date of users

(map (fn [user] (user :member-since)) users)

("2009-04-16" "2009-02-01" "2009-03-30")

In [49]:
; Using the # reader macro as a shortcut for anonymous functions

(map #(% :member-since) users) ; The anonymous function is #(% :member-since)


("2009-04-16" "2009-02-01" "2009-03-30")

Applying anonymous functions to sequences of variable length


In [57]:
(#(vector %&) 1 2 3 4 5)

[(1 2 3 4 5)]

In [58]:
(#(vector %1 %&) 1 2 3 4 5)

[1 (2 3 4 5)]

In [59]:
(#(vector %1 %2 %&) 1 2 3 4 5)

[1 2 (3 4 5)]

In [60]:
(#(vector %1 %2 %&) 1 2)

[1 2 nil]